Шепелев Д.

# **Моделирование языка на уровне символов**

**В качестве входных данных я выбрал текст книги "451 градус по фаренгейту", который сохранил в .txt-формате**

## **Импорты**

In [3]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

## **Чтение и обработка текста**

In [4]:
with open('451_full.txt', 'r', encoding="utf8") as fp:
    text = fp.read()
    
start_index = text.find('Если тебе дадут линованную бумагу, пиши поперек.')
end_index = text.find('~~~ Конец ~~~')

text = text[start_index:end_index]
char_set = set(text)

print(f'Общая длина: {len(text)}')
print(f'Уникальные символы: {len(char_set)}')

Общая длина: 270915
Уникальные символы: 88


### **Кодирование текста**

Определим вспомогательные функции

In [8]:
def char2int(char_sorted) -> dict[str,int]:
    return {ch:i for i,ch in enumerate(char_sorted)}

In [9]:
def text2int(text, char_encoded) -> np.ndarray[int]:
    return np.array([char_encoded[ch] for ch in text], dtype=np.int32)

In [10]:
def int2text(text_encoded, char_array) -> str:
    return ''.join(char_array[text_encoded]) 

In [22]:
char_sorted = sorted(char_set)
char_encoded = char2int(char_sorted)
char_array = np.array(char_sorted)

encoding_test_sample = text2int(text[:48], char_encoded)
print(f'Закодированное предложение: \n{encoding_test_sample}')
print()
decoding_test_sample = int2text(encoding_test_sample, char_array)
print(f'Декодированное предложение: \n{decoding_test_sample}')

Закодированное предложение: 
[28 67 61 58  1 68 55 51 55  1 54 50 54 69 68  1 61 58 63 64 52 50 63 63
 69 80  1 51 69 62 50 53 69  6  1 65 58 74 58  1 65 64 65 55 66 55 60  8]

Декодированное предложение: 
Если тебе дадут линованную бумагу, пиши поперек.


In [23]:
text_encoded = text2int(text, char_encoded)

print(f'Размерность закодированного текста: {text_encoded.shape}')

Размерность закодированного текста: (270915,)


## **Формирование датасета**

In [27]:
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for example in ds_text_encoded.take(10):
    print(f'{example.numpy()} -> {char_array[example.numpy()]}')

28 -> Е
67 -> с
61 -> л
58 -> и
1 ->  
68 -> т
55 -> е
51 -> б
55 -> е
1 ->  


Определм функцию для формирования `x` и `y`. 
- `x` - уже сгенерированные символы
- `y` - следующий необходимый символ

In [28]:
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

In [29]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)
ds_sequences = ds_chunks.map(split_input_target)

## **Разделение данных на мини-пакеты**

In [30]:
tf.random.set_seed(451)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

for example in ds.take(1):
    print(f'Вход Х:\n{example[0]}')
    print(f'Цель Y:\n{example[1]}')

Вход Х:
[[62  1 67 ... 65 66 64]
 [68 66 55 ... 61 58  8]
 [66 64 56 ... 60 63 58]
 ...
 [58 72 64 ...  1 51 61]
 [61 80 54 ... 58 53 58]
 [ 8  1 27 ... 55 66 68]]
Цель Y:
[[ 1 67 60 ... 66 64 67]
 [66 55 52 ... 58  8 20]
 [64 56 50 ... 63 58 53]
 ...
 [72 64 19 ... 51 61 58]
 [80 54 81 ... 53 58  1]
 [ 1 27 66 ... 66 68 50]]


## **Построение модели на основе RNN**

In [40]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, embedding_dim),
        layers.LSTM(rnn_units, return_sequences=True),
        layers.Dense(vocab_size)
    ])
    return model

## **Установка параметров обучения**

In [41]:
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(451)
model = build_model(vocab_size=charset_size, 
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units)

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 256)         22528     
                                                                 
 lstm_1 (LSTM)               (None, None, 512)         1574912   
                                                                 
 dense_1 (Dense)             (None, None, 88)          45144     
                                                                 
Total params: 1,642,584
Trainable params: 1,642,584
Non-trainable params: 0
_________________________________________________________________


## **Обучение**

In [43]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [44]:
history = model.fit(ds, epochs=40)

Epoch 1/40
104/104 [==============================] - 25s 221ms/step - loss: 3.1976
Epoch 2/40
104/104 [==============================] - 23s 223ms/step - loss: 2.5570
Epoch 3/40
104/104 [==============================] - 24s 224ms/step - loss: 2.3810
Epoch 4/40
104/104 [==============================] - 23s 223ms/step - loss: 2.2609
Epoch 5/40
104/104 [==============================] - 23s 223ms/step - loss: 2.1663
Epoch 6/40
104/104 [==============================] - 23s 222ms/step - loss: 2.0851
Epoch 7/40
104/104 [==============================] - 23s 222ms/step - loss: 2.0115
Epoch 8/40
104/104 [==============================] - 23s 223ms/step - loss: 1.9477
Epoch 9/40
104/104 [==============================] - 23s 223ms/step - loss: 1.8899
Epoch 10/40
104/104 [==============================] - 23s 223ms/step - loss: 1.8389
Epoch 11/40
104/104 [==============================] - 24s 224ms/step - loss: 1.7921
Epoch 12/40
104/104 [==============================] - 24s 224ms/step - lo

## **Генерация новых отрывков текста**

In [46]:
def sample(model, starting_str, len_generated_text=500, max_input_length=40, scale_factor=1.0):
    encoded_input = text2int(starting_str, char_encoded)
    encoded_input = tf.reshape(encoded_input, (1,-1))
    
    generated_str = starting_str
    
    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)
        
        scaled_logits = logits * scale_factor
        new_char_index = tf.random.categorical(scaled_logits, num_samples=1)
        
        new_char_index = tf.squeeze(new_char_index)[-1].numpy()
        
        generated_str += str(char_array[new_char_index])
        
        new_char_index = tf.expand_dims([new_char_index], 0)
        encoded_input = tf.concat([encoded_input, new_char_index], axis=1)
        encoded_input = encoded_input[:, -max_input_length:]
        
    return generated_str

In [51]:
tf.random.set_seed(451)
print(sample(model, 'Они сжигали книги ', scale_factor=1.0))

Они сжигали книги и испорающие своей мягко самого пять задевать с ней снять.
— Я повершен с вами имече, — я говорит: «Мне не было эту ковида. Качертает. Мы словно не «солнее нофью из-за усового обошлом. Воздух огнев земле.
— Чудст! Не металла, скряды? — Брандме»се» в один поглядым стоблению.
Надвое! Это знает ряде, чтобы и не присятся с глаза от бран двейной тишине, словно на книгу по мерторга:
— Монтэг, я почувствиева, началось, вы — это нельзя»! Я и сего на это сделать.
— Вы знаете, что, вы не виноватым, потом 


In [57]:
tf.random.set_seed(451)
print(sample(model, 'Они сжигали книги ', scale_factor=5.0))

Они сжигали книги и грохотала из грубоко взгляды, на котором поджигалия, как теперь все это заборлить и окончилась война. Она подошел к вам нога. Он поднял с ней под конце конценцими, скрывающихся в темноту, не слова не задумаю свои коснулся его сквозь эту неделю по кругу в ваше двое и смотреть, как он стоял в темноте по домом, как они видел ноги, о вашем мне по улицу волосы, он остановился на полу. Он несколько раз повернул и опустился на стену, где можно было сохнать и все станет свои грубоко засунул руку краск


In [53]:
tf.random.set_seed(451)
print(sample(model, 'Они сжигали книги ', scale_factor=0.5))

Они сжигали книги и их ды ющу ригодцвой корп«седои, — кигивает рубки.
— вряд рошки.
Заучи те: Что и теать!» — когда»-Громой маюрь, —ы, Фаба видеіь! И говорю, чуть ли 'дрей тогу рядоко, Гослуш.
Ее золщное стены, как Клари.
Вопья: в резан самом бежня я щип..
Онта, ичел. Млежке еслил съе.
Стоки! Я птил, чтобы здарыт Тониющую голустурыр фаигнав запом. Ко вбив, лишиний ничнастлизу детел, — отлиюте, Шеспроси, чьи-куру, а?
Ктара — только раношэ… Он влоя?
— Видите ражьзя какой-то щекс — будто чтояь уностейших, станупый, 
